In [15]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import joblib

from data_preprocessing import pipeline
from feature_preprocessing import pipeline2

### 데이터 전처리

In [ ]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스2.csv"
# df = pd.read_csv(file_path, encoding ='cp949')
df = pd.read_csv(file_path)

# 데이터 전처리````
preprocessing_pipeline = pipeline()
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "250318-03-삭제예정"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_18176\4291326278.py:4: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
c:\Users\user\mid_project\src\data_preprocessing.py:122: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_price = pd.read_csv(
c:\Users\user\mid_project\src\data_preprocessing.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['최저당첨가점','최고당첨가점', '평균당첨가점']].fillna(0, inplace=True)


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2981 entries, 44 to 17246
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공급지역코드        2981 non-null   int64  
 1   공급규모          2981 non-null   int64  
 2   투기과열지구        2981 non-null   object 
 3   조정대상지역        2981 non-null   object 
 4   분양가상한제        2981 non-null   object 
 5   정비사업          2981 non-null   object 
 6   공공주택지구        2981 non-null   object 
 7   대규모택지개발지구     2981 non-null   object 
 8   수도권내민영공공주택지구  2981 non-null   object 
 9   공급세대수         2981 non-null   int64  
 10  순위            2981 non-null   object 
 11  거주지역          2981 non-null   object 
 12  접수건수          2981 non-null   int64  
 13  경쟁률           2981 non-null   float64
 14  최저당첨가점        2981 non-null   float64
 15  최고당첨가점        2981 non-null   float64
 16  평균당첨가점        2981 non-null   float64
 17  토픽 1          2981 non-null   float64
 18  토픽 2          2981 non-null   f

### 모델 학습 및 평가

In [18]:
# 전처리된 데이터 가져오기

# file_version = "250317-01"
# file_path = f"./storage/train_data/t``ain-{file_version}.csv"
# df = pd.read_csv(file_path, encoding='cp949')

df = pd.read_csv("C:/Users/user/mid_project/src/storage/train_data/train-250318-03-삭제예정.csv", encoding = 'cp949')


In [19]:
X = df.drop(columns=["최저당첨가점"])
Y = df["최저당첨가점"]

X_train, X_test, y_train, y_test = train_test_split(
    X,  # 타겟(최저당첨가점) 제외
    Y,  # 타겟
    test_size=0.2,
    random_state=42,
    stratify=df["투기과열지구"]  # 분포 맞추기 위해 미달여부로 stratify
)

# 분양가상한제, 투기과열지구, 조정대상지역 -> 고려해보기염

In [20]:
feature_pipeline = pipeline2()

# 학습 데이터(X_train)에 fit
feature_pipeline.fit(X_train)

# 학습 데이터(X_train)를 변환
X_train_transformed = feature_pipeline.transform(X_train)

# 테스트 데이터(X_test)를 변환
X_test_transformed = feature_pipeline.transform(X_test)

In [21]:
X_train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2384 entries, 2881 to 1818
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   공급규모            2384 non-null   float64
 1   공급세대수           2384 non-null   float64
 2   접수건수            2384 non-null   float64
 3   경쟁률             2384 non-null   float64
 4   최고당첨가점          2384 non-null   float64
 5   평균당첨가점          2384 non-null   float64
 6   토픽 1            2384 non-null   float64
 7   토픽 2            2384 non-null   float64
 8   토픽 3            2384 non-null   float64
 9   토픽 4            2384 non-null   float64
 10  토픽 5            2384 non-null   float64
 11  토픽 6            2384 non-null   float64
 12  토픽 7            2384 non-null   float64
 13  전용면적            2384 non-null   float64
 14  공급금액(최고가 기준)    2384 non-null   float64
 15  전용면적당 시세차익      2075 non-null   float64
 16  투기과열지구_N        2384 non-null   bool   
 17  투기과열지구_Y        2384 non-null   boo

In [22]:
# TODO: 여러 모델 학습 및 하이퍼파라미터 최적화

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

# 모델 학습
model.fit(X_train_transformed, y_train)

# 예측
y_pred = model.predict(X_test_transformed)

# 평가 (RMSE와 R^2 계산)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse) # 낮을수록 좋음
print("R^2 Score:", r2) # 1에 가까울수록 좋음

RMSE: 2.7122038854495996
R^2 Score: 0.9671236072463428


### 모델 저장 

In [14]:
# 모델 저장 및 로드

version = '0.0.2'

# 모델 저장
joblib.dump(model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_0.0.2.pkl']

In [8]:
# 모델 로드
loaded_model = joblib.load("./storage/trained_model/model_0.0.1.pkl")

# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)